In [1]:
from utils import plot_point_and_box

plot_point_and_box(point=(750, 500), box=(750, 500, 300, 200))

/rhome/zmirz004/bigdata/.conda/envs/sam3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from utils import run_sam3_segmentation_pipeline

frames_dir = 'frames'

output_video_path, video_segments = run_sam3_segmentation_pipeline(
    frames_dir=frames_dir,
    input_point=(275, 768),
    output_video_path='output1.mp4'
)

Initializing SAM 3 on cuda...


INFO 2025-12-04 00:17:39,544 1148509 sam3_video_base.py: 124: setting max_num_objects=10000 and num_obj_for_compile=16
frame loading (JPEG): 100%|██████████| 916/916 [00:57<00:00, 15.90it/s]


Propagating masks...


propagate in video: 100%|██████████| 916/916 [00:56<00:00, 16.13it/s]


Overlay video written to: output1.mp4


In [ ]:
from utils import sam2_point_track_and_overlay
from definitions import ROOT_DIR
import os




frame loading (JPEG): 100%|██████████| 916/916 [00:53<00:00, 17.04it/s]
/rhome/zmirz004/bigdata/.conda/envs/sam3/lib/python3.12/site-packages/sam2/sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (/rhome/zmirz004/bigdata/.conda/envs/sam3/lib/python3.12/site-packages/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 916/916 [00:33<00:00, 26.96it/s]


Wrote: overlay_supervision.mp4
